In [1]:
cd /data/store01/data_dirs/students/ucfafyi/S2_MODIS

/data/store01/data_dirs/students/ucfafyi/S2_MODIS


In [7]:
import os
import sys
sys.path.insert(0,'python')
import multiprocessing
from fft_helper import *
from functools import partial
import multiprocessing
import gdal
from L_geo import *
from get_brdf import *
from geo_trans import *
from Dload_Finder import file_finder
import datetime
import glob
from cloud import get_cloud_mask
from classification import *
import multiprocessing
from scipy import ndimage, signal, optimize
import scipy
from get_wrs import *
directory = os.getcwd()+'/'
from smoothn import *
import tifffile

def fun(f, q_in, q_out):
    while True:
        i, x = q_in.get()
        if i is None:
            break
        q_out.put((i, f(x)))


def parmap(f, X, nprocs=multiprocessing.cpu_count()):
    q_in = multiprocessing.Queue(1)
    q_out = multiprocessing.Queue()

    proc = [multiprocessing.Process(target=fun, args=(f, q_in, q_out))
            for _ in range(nprocs)]
    for p in proc:
        p.daemon = True
        p.start()

    sent = [q_in.put((i, x)) for i, x in enumerate(X)]
    [q_in.put((None, None)) for _ in range(nprocs)]
    res = [q_out.get() for _ in range(len(sent))]

    [p.join() for p in proc]

    return [x for i, x in sorted(res)]



class PSF_optimization(object):
    def __init__(self, year, month, day, doy, lat, lon, sate):
        self.year = year
        self.doy = doy
        self.month = month
        self.day= day
        self.lat = lat
        self.lon = lon
        self.sate = sate 
        self.Lx = None
        self.LY = None
        self.Hx = None
        self.Hy = None
        self.H_data = None
        self.L_data = None
        self.sza = None
        self.saa = None
        self.vza = None
        self.vaa = None
        self.retval = None
        self.BRDF_16_days =None
        self.composite_brdf = None
        self.base_mask = None
        
    def S2_PSF_optimization(self):
        
        self.h,self.v = mtile_cal(self.lat, self.lon)
        m = mgrs.MGRS()
        mg_coor = m.toMGRS(self.lat, self.lon, MGRSPrecision=4)
        self.place = mg_coor[:5]
        #self.Hfiles = glob.glob(directory +'l_data/LC8%03d%03d%d*LGN00_sr_band1.tif'%(self.path, self.row, self.year))
        self.Hfile = os.getcwd()+'/s_data/%s/%s/%s/%d/%d/%d/0/'%(mg_coor[:2], mg_coor[2], mg_coor[3:5], self.year, self.month, self.day)
        #Lfile = glob.glob('m_data/MCD43A1.A%d%03d.h%02dv%02d.006.*.hdf'%(year,doy,h,v))[0]
        self.doy = datetime .datetime(self.year, self.month, self.day).timetuple().tm_yday
        self.Lfiles = [glob.glob('m_data/MCD43A1.A%d%03d.h%02dv%02d.006.*.hdf'%(self.year,i,self.h,self.v))[0] for i in range(self.doy-8, self.doy+9)]
        
        if glob.glob(self.Hfile+'cloud.tif')==[]:
            cl = classification(fhead = self.Hfile, bands = (2,3,4,8,11,12,13), bounds = None)
            cl.Get_cm_p()
            self.cloud = cl.cm.copy()
            tifffile.imsave(self.Hfile+'cloud.tif', self.cloud)
            self.H_data = np.repeat(np.repeat(cl.b12, 2, axis=1), 2, axis=0)
            del cl
        else:
            self.cloud = tifffile.imread(self.Hfile+'cloud.tif')
        cloud_cover = 1.*self.cloud.sum()/self.cloud.size
        if cloud_cover > 0.2:  
            print 'Too much cloud, cloud proportion: %.03f !!'%cloud_cover
        else:
            mete = readxml('%smetadata.xml'%self.Hfile)
            self.sza = np.zeros(7)
            self.sza[:] = mete['mSz']
            self.saa = self.sza.copy()
            self.saa[:] = mete['mSa']
            self.vza = (mete['mVz'])[[1,2,3,7,8,11,12],]
            self.vaa = (mete['mVa'])[[1,2,3,7,8,11,12],]
            
            self.L_inds, self.H_inds = get_coords(self.lat,self.lon)
            self.Lx, self.Ly = self.L_inds
            self.Hx, self.Hy = self.H_inds

            angles = (self.sza[-1], self.vza[-1], (self.vaa - self.saa)[-1])

            self.BRDF_16_days = np.array([get_brdf_six(Lfile,angles,bands=(7,), \
                                              flag=None, Linds= self.L_inds) for Lfile in self.Lfiles]).squeeze()

            valid_range = (self.BRDF_16_days[:,0,:]>=0)&(self.BRDF_16_days[:,0,:]<=1)
            magic = 0.618034
            test = self.BRDF_16_days[:,0,:].copy()
            test[~valid_range]=np.nan
            W = magic**self.BRDF_16_days[:,1,:]
            W[self.BRDF_16_days[:,1,:]>1]=0
            
            smothed = smoothn(test, axis=0, isrobust=1, W =W, s=1)[0]
            
            
            struct = ndimage.generate_binary_structure(2, 2)
            dia_cloud = ndimage.binary_dilation(self.cloud, structure=struct, iterations=60).astype(self.cloud.dtype)

            mask = ~(self.H_data<=0).astype('bool')
            small_mask = ndimage.binary_erosion(mask, structure=struct, iterations=60).astype(mask.dtype)
            self.val_mask = (~dia_cloud)&small_mask

            avker = np.ones((120,120))
            navker = avker/avker.sum()
            self.s = signal.fftconvolve(self.H_data, navker, mode='same')
            self.s[~self.val_mask]=np.nan

            self.L_data = np.zeros(self.BRDF_16_days[8,0,:].shape[0])
            self.L_data[:] = np.nan
            self.L_data[self.BRDF_16_days[8,1,:]==0] = smothed[8,:][self.BRDF_16_days[8,1,:]==0]
            #args = s, self.L_data, 

            xstd,ystd = np.mgrid[-30:30:2, -30:30:2]
            ps = zip(xstd.ravel(), ystd.ravel())

            self.solved = parmap(self.op1, ps, nprocs=32)

            return self.solved, self.BRDF_16_days, smothed
                        
    def _cost1(self, shifts):
        xs, ys = shifts 
        val = (self.Hx+xs<self.s.shape[0])&(self.Hy+ys<self.s.shape[1])
        shx, shy = (self.Hx+xs).astype(int), (self.Hy+ys).astype(int)
        Lvals, Hvals = self.L_data[val], self.s[shx[val], shy[val]]
        Lvals[np.isnan(Lvals)],Hvals[np.isnan(Hvals)]=-9999999, -9999999
        mas = (Lvals>0)&(Lvals<1)&(Hvals>0)&(Hvals<1)
        r = scipy.stats.linregress(Lvals[mas], Hvals[mas])
        return abs(1-r.rvalue)

    def gaussian(self, xstd, ystd, angle, norm = True):
        win = int(round(max(3*xstd, 3*ystd)))
        winx = win*2**0.5
        winy = win*2**0.5

        xgaus = signal.gaussian(winx, xstd)
        ygaus = signal.gaussian(winy, ystd)
        gaus  = np.outer(xgaus, ygaus)
        r_gaus = ndimage.interpolation.rotate(gaus, angle, reshape=True)
        center = np.array(r_gaus.shape)/2
        cgaus = r_gaus[center[0]-win/2: center[0]+win/2, center[1]-win/2:center[1]+win/2]
        if norm:
            return cgaus/cgaus.sum()
        else:
            return cgaus 

    def _cost2(self, para):
        xstd,ystd,angle, xs, ys = para 
        G = self.gaussian(xstd,ystd,angle,True)                              
        ss = signal.fftconvolve(self.H_data, G, mode='same')
        # remove the cloud pixel
        ss[~self.val_mask]=np.nan
        val = (self.Hx+xs<self.H_data.shape[0])&(self.Hy+ys<self.H_data.shape[1])
        shx, shy = (self.Hx+xs).astype(int), (self.Hy+ys).astype(int)
        Lvals, Hvals = self.L_data[val], ss[shx[val], shy[val]]
        Lvals[np.isnan(Lvals)],Hvals[np.isnan(Hvals)]=-9999999, -9999999
        mas = (Lvals>0)&(Lvals<1)&(Hvals>0)&(Hvals<1)
        r = scipy.stats.linregress(Lvals[mas], Hvals[mas])
        costs = abs(1-r.rvalue)
        return costs 

    def op1(self, p0):
        #p0 =  ps[ind
        #args = self.H_data, self.L_data, self.val_mas
        return optimize.fmin(self._cost1, p0, full_output=1, maxiter=250, maxfun=250)

    

In [5]:
import os
import sys
sys.path.insert(0,'python')
from fastRWpkl import *
from get_wrs import *
from linear_plot import *
from smoothn import *
plot_config()
import pylab as plt
%matplotlib inline
#from glob import glob
import mgrs

/home/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/home/ucfafyi/.conda/envs/gdal_env/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: svg.embed_char_paths is deprecated and replaced with svg.fonttype; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [ ]:
files = '''50SMH - 2016-11-16
50SMH - 2016-11-06
50SMH - 2016-09-07
50SMH - 2016-08-28
50SMH - 2016-07-29
50SMH - 2016-05-10
50SMH - 2016-04-30
50SMH - 2016-04-10
50SMH - 2016-02-10'''
lat, lon = 38.84313441, 115.84771239 
fnames = sorted([[i.split(' - ')[0], i.split(' - ')[1]] for i in files.split('\n')])
m = mgrs.MGRS()
retval = []
for fi in fnames:
    pos, date = fi
    da = datetime.datetime.strptime(date, '%Y-%m-%d')
    doy = datetime .datetime(da.year, da.month, da.day).timetuple().tm_yday
    lat, lon = m.toLatLon(pos)
    if da.month<=10:
        C = PSF_optimization(da.year, da.month, da.day, doy, lat+0.5, lon+1, 'S2')
        data = C.S2_PSF_optimization()
        #paras = np.array([i[0] for i in data[0]])
        #costs = np.array([i[1] for i in data[0]])
        #op_pa = paras[costs==costs.min()]
        retval.append([fi,data])

In [ ]:
lat, lon = 37.94208532, 115.86192607

In [ ]:
fnames = sorted([[i.split(' - ')[0], i.split(' - ')[1]] for i in files.split('\n')])

In [ ]:
fnames

In [ ]:
m = mgrs.MGRS()
retval = []
for fi in fnames:
    pos, date = fi
    da = datetime.datetime.strptime(date, '%Y-%m-%d')
    doy = datetime .datetime(da.year, da.month, da.day).timetuple().tm_yday
    lat, lon = m.toLatLon(pos)
    if da.month<=10:
        try:
            C = PSF_optimization(da.year, da.month, da.day, doy, lat+0.5, lon+1, 'S2')
            data = C.S2_PSF_optimization()
            paras = np.array([i[0] for i in data[0]])
            costs = np.array([i[1] for i in data[0]])
            op_pa = paras[costs==costs.min()]
            retval.append([fi, op_pa, costs.min()])
        except:
            pass

In [ ]:
L_data, s, Hx, Hy, Lx, Ly = C.L_data, C.s, C.Hx, C.Hy, C.Lx, C.Ly

In [47]:
def _cost1(shifts):
        xs, ys = shifts 
        val = (Hx+xs<s.shape[0])&(Hy+ys<s.shape[1])
        shx, shy = (Hx+xs).astype(int), (Hy+ys).astype(int)
        Lvals, Hvals = L_data[val], s[shx[val], shy[val]]
        Lvals[np.isnan(Lvals)],Hvals[np.isnan(Hvals)]=-9999999, -9999999
        mas = (Lvals>0)&(Lvals<1)&(Hvals>0)&(Hvals<1)
        r = scipy.stats.linregress(Lvals[mas], Hvals[mas])
        return abs(1-r.rvalue)
def op1(p0):
        #p0 =  ps[ind]
        #args = self.H_data, self.L_data, self.val_mask
        return optimize.fmin(_cost1, p0, full_output=1, maxiter=250, maxfun=250)